In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import scipy
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

params = {'legend.fontsize': 20,
          'figure.figsize': (15, 10),
         'axes.labelsize': 20,
         'axes.titlesize': 20,
         'xtick.labelsize': 20,
         'ytick.labelsize': 20}
pylab.rcParams.update(params)
plt.close('all')

## Files are stored in the '2015' and '2016' subdirectories, relative to the working directory
path2015 = Path('../2015')
path2016 = Path('../2016')
dp = np.linspace(0,100,11)
dy = np.linspace(1945,2020,15)
pd.options.display.max_columns = None


In [ ]:
## Column names in the 2016 version of the Physician Compare National Downloadable File are simple and coding-friendly 
## except for a leading space in all columns after the first. That file is read first and its column names (with leading 
## space removed) used to replace column names of the 2015 version data after its file is read. Numerical columns are read 
## as floats (many have NaNs), except where that failed due to presence of a string. As of now, these are handled, when 
## necessary, on a case-by-case basis. ndf16.dtypes.equals(ndf15.dtypes) returns True.

col_names16 = pd.read_csv(path2016 / 'Physician_Compare_National_Downloadable_File.csv', nrows=0).columns
types_dict = {"NPI": float, " Ind_PAC_ID": float, " Grd_yr": float, " org_pac_id": float, " num_org_mem": float}
types_dict.update({col: str for col in col_names16 if col not in types_dict})
ndf16 = pd.read_csv(path2016 / 'Physician_Compare_National_Downloadable_File.csv', dtype=types_dict).rename(columns=lambda x: x.strip())

'''col_names15 = pd.read_csv(path2015 / 'Physician_Compare_National_Downloadable_File.csv', nrows=0).columns
types_dict = {"NPI": float, "PAC ID": float, "Graduation year": float, "Group Practice PAC ID": float, "Number of Group Practice members": float}
types_dict.update({col: str for col in col_names15 if col not in types_dict})
ndf15 = pd.read_csv(path2015 / 'Physician_Compare_National_Downloadable_File.csv', dtype=types_dict)
ndf15.rename(columns=dict(zip(ndf15.columns.values, ndf16.columns.values)), inplace=True)

## The 2016 version of the Individual physician measured performance rates contains a new column not found in the 2015 version,
## patient count. A patient_count column containing float zeros is added to the 2015 data in matching position to where it is
## ips16.dtypes.equals(ips15.dtypes) returns True.
'''
col_names16 = pd.read_csv(path2016 / 'Physician_Compare_2016_Individual_EP_Public_Reporting.csv', nrows=0).columns
types_dict = {"NPI": float, " Ind_PAC_ID": float, " prf_rate": float, " patient_count": float}
types_dict.update({col: str for col in col_names16 if col not in types_dict})
ips16 = pd.read_csv(path2016 / 'Physician_Compare_2016_Individual_EP_Public_Reporting.csv', dtype=types_dict).rename(columns=lambda x: x.strip())

'''col_names15 = pd.read_csv(path2015 / 'Physician_Compare_2015_Individual_EP_Public_Reporting___Performance_Scores.csv', nrows=0).columns
types_dict = {"NPI": float, "PAC ID": float, "Measure Performance Rate": float}
types_dict.update({col: str for col in col_names15 if col not in types_dict})
ips15 = pd.read_csv(path2015 / 'Physician_Compare_2015_Individual_EP_Public_Reporting___Performance_Scores.csv', dtype=types_dict)
ips15.insert(loc=8, column='patient_count', value=0)
ips15.rename(columns=dict(zip(ips15.columns.values, ips16.columns.values)), inplace=True)

## The Clinician Utilization data is new for 2016.
col_names16 = pd.read_csv(path2016 / 'Physician_Compare_Clinician_Utilization_Data.csv', nrows=0).columns
types_dict = {"NPI": float, " Ind_PAC_ID": float, " line_srvc_cnt": float, " bene_cnt": float}
types_dict.update({col: str for col in col_names16 if col not in types_dict})
cud16 = pd.read_csv(path2016 / 'Physician_Compare_Clinician_Utilization_Data.csv', dtype=types_dict).rename(columns=lambda x: x.strip())
'''

In [ ]:
ips16.head()

# Calculation of PQRS "participation rate", unique providers with PQRS == 'Y' to unique providers (PQRS either 'Y' or NaN)

In [3]:
# Calculation of PQRS "participation rate", unique providers with PQRS == 'Y' to unique providers (PQRS either 'Y' or NaN)

ndf16trunc = ndf16.loc[:, 'NPI':'pri_spec'].drop_duplicates()
ndf16trunc_pqrs = ndf16.loc[:, ['NPI', 'Ind_PAC_ID', 'lst_nm', 'frst_nm', 'Med_sch', 'Grd_yr', 'pri_spec', 'PQRS']].drop_duplicates()

pqrs_y = ndf16trunc_pqrs.loc[ndf16trunc_pqrs.PQRS == 'Y', :].PQRS.count()
pqrs_n = ndf16trunc_pqrs.PQRS.isna().count()
participation_rate_overall = pqrs_y/(pqrs_y + pqrs_n)
print(participation_rate_overall)



0.37038767962831654


In [8]:
num_practitioners_per_measure = ips16.measure_title.value_counts()

num_practitioners_per_measure.to_csv('../num_practitioners_per_measure.csv', index=True)
#measure_names_descending_frequency = ips16.measure_title.value_counts().index

#measure_names_descending_frequency_list = ips16.measure_title.value_counts().index.to_list()
#ndf16trunc.loc[:, 'pri_spec'].value_counts().to_csv('../practitoners_per_specialty.csv', index=True)
num_practitioners_per_specialty = ndf16trunc.loc[:, 'pri_spec'].value_counts()
type(num_practitioners_per_measure.to_frame())
num_practitioners_per_measure.to_frame

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


<bound method Series.to_frame of Documentation of Current Medications in the Medical Record                                                                    64975
Preventive Care and Screening: Tobacco Use: Screening and Cessation Intervention                                              40383
Pain Assessment and Follow-Up                                                                                                 34890
Functional Outcome Assessment                                                                                                 26326
Pneumonia Vaccination Status for Older Adults                                                                                 18801
Preventive Care and Screening: Influenza Immunization                                                                         16161
Colorectal Cancer Screening                                                                                                   14463
Advance Care Plan                          

In [ ]:
pd.qcut(ips16.NPI, 86)
num_practitioners_per_measure.index[0]
ips16.measure_title == num_practitioners_per_measure.index[0]

In [29]:
for idx, row in enumerate(num_practitioners_per_measure.index):
    print(ips16.loc[ips16.measure_title == row, :].patient_count.sum())

#    print(row)

#num_patients_top_measure = ips16.loc[ips16.measure_title == num_practitioners_per_measure.index[0],:].patient_count.sum()
#num_patients_top_measure

#def f(x):
#    return ips16.loc[ips16.measure_title == x, :].patient_count



#num_practitioners_per_measure.to_frame().apply(f)

#ips16.loc[ips16.measure_title == num_practitioners_per_measure.to_frame(), :].patient_count.head()
#num_practitioners_per_measure.to_frame()

23099461.0
9846012.0
10945338.0
6729621.0
3610827.0
2716602.0
1833340.0
2466976.0
4035150.0
984763.0
683333.0
871024.0
2837240.0
242492.0
446060.0
173433.0
193465.0
125550.0
105121.0
176717.0
140857.0
131710.0
885266.0
139067.0
837992.0
803231.0
119015.0
228129.0
114665.0
65641.0
79116.0
580244.0
26732.0
32612.0
34813.0
308880.0
212693.0
29647.0
43232.0
16673.0
20602.0
11520.0
182786.0
10966.0
19950.0
8967.0
4427255.0
5391.0
12171.0
18036.0
11487.0
8151.0
10786.0
12555.0
10243.0
25018.0
10948.0
3850.0
3936.0
23141.0
3635.0
5646.0
17664.0
2418.0
1389.0
2673.0
10352.0
30483.0
30483.0
3647.0
7414.0
16315.0
12418.0
8224.0
129932.0
10052.0
6425.0
24616.0
5581.0
4999.0
10176.0
53450.0
1090.0
3094.0
3286.0
11343.0


In [ ]:

num_patients_top_measure = ips16.loc[ips16.measure_title == num_practitioners_per_measure.index[0],:].patient_count.sum()
num_patients_top_measure

#ips16.loc[ips16.measure_title == num_practitioners_per_measure.index[50], 'patient_count'].sum()
#ips16.loc[ips16.measure_title == num_practitioners_per_measure.index[0],:].patient_count.sum()

In [ ]:
#isin(values)	Whether each element in the DataFrame is contained in values.


In [ ]:
num_practitioners_per_measure.head()

In [ ]:
num_unique_providers_in_ndf = ndf16t.NPI.drop_duplicates().count()
num_unique_providers_in_ips = ips16.NPI.drop_duplicates().count()
num_patients_treated = ips16.patient_count.sum()
num_measures = ips16.measure_title.drop_duplicates().count()
num_phys_specialties = ndf16t.pri_spec.drop_duplicates().count()

num_unique_providers_in_ndf, num_unique_providers_in_ips, num_patients_treated, num_measures, num_phys_specialties

In [ ]:
ips16.measure_title.drop_duplicates().count()

In [ ]:
phys_specialties = ndf16t.pri_spec.drop_duplicates()
med_schools = ndf16t.Med_sch.drop_duplicates()
measure_num_name_pairs = ips16[['measure_ID', 'measure_title']].drop_duplicates()
measure_num_name_pairs

#med_schools.drop_duplicates().to_csv('../med_sch.csv', index=False)
#measure_num_name_pairs.drop_duplicates().to_csv('../measuretitles.csv', index=False)
#phys_specialties.drop_duplicates().to_csv('../phys_specialties.csv', index=False)

#phys_specialties
#ips16[ips16['NPI','measure_ID']]

# Figure 1

In [ ]:
dy = np.linspace(1945,2020,16)
a16 = pd.value_counts(ndf16.loc[(ndf16['PQRS']=='Y'),['NPI','Grd_yr']].drop_duplicates().dropna().Grd_yr, bins=dy, sort=False)
b16 = pd.value_counts(ndf16[['NPI','Grd_yr']].drop_duplicates().dropna().Grd_yr, bins=dy, sort=False)
c16 = a16/b16
ax = c16.plot.bar(ylim=(0,0.7), title='Clinician Participation Rate vs Year of Terminal Degree (2016 Data)')
ax.set(xlabel="Year of Clinician's Terminal Degree", ylabel="Clinician participation rate")

# Figure 2

In [ ]:
a16 = pd.value_counts(ndf16.loc[(ndf16['PQRS']=='Y'),['NPI','Grd_yr']].drop_duplicates().dropna().Grd_yr, bins=dy, sort=False)
b16 = pd.value_counts(ndf16[['NPI','Grd_yr']].drop_duplicates().dropna().Grd_yr, bins=dy, sort=False)
c16 = a16/b16
ax = c16.plot.bar(ylim=(0,0.7), title='Clinician Participation Rate vs Year of Terminal Degree (2016 Data)')
ax.set(xlabel="Year of Clinician's Terminal Degree", ylabel="Clinician participation rate")

# Figure 3

In [ ]:
ax = (c16/c15).iloc[0:13].plot.bar(ylim=(0.9,1.1), title='Participation rate change vs year of terminal degree (2016 data vs 2015 data)')
ax.set(xlabel="Year of Clinician's Terminal Degree", ylabel="Clinician participation rate year over year change")
#print((c16/c15).iloc[0:13])

In [ ]:
## This code section examines Pneumonia Vaccination Status for Older Adults
## Need to explore weighting performance scores by patient count for 2016 data, will be interesting to contrast with the 2015 
## data where patient count is not available

a = ips16.loc[(ips16['measure_ID']=='PQRS_EP_111_1'),['NPI','prf_rate','patient_count']].drop_duplicates().dropna()

b1 = ndf16.loc[(ndf16['pri_spec']=='INTERNAL MEDICINE'),['NPI']].drop_duplicates().dropna()
c1 = pd.merge(a, b1, on='NPI')

b2 = ndf16.loc[(ndf16['pri_spec']=='INFECTIOUS DISEASE'),['NPI']].drop_duplicates().dropna()
c2 = pd.merge(a, b2, on='NPI')

b3 = ndf16.loc[(ndf16['pri_spec']=='EMERGENCY MEDICINE'),['NPI']].drop_duplicates().dropna()
c3 = pd.merge(a, b3, on='NPI')

b4 = ndf16.loc[(ndf16['pri_spec']=='PULMONARY DISEASE'),['NPI']].drop_duplicates().dropna()
c4 = pd.merge(a, b4, on='NPI')

b5 = ndf16.loc[(ndf16['pri_spec']=='NURSE PRACTITIONER'),['NPI']].drop_duplicates().dropna()
c5 = pd.merge(a, b5, on='NPI')

b6 = ndf16.loc[(ndf16['pri_spec']=='MEDICAL ONCOLOGY'),['NPI']].drop_duplicates().dropna()
c6 = pd.merge(a, b6, on='NPI')

b7 = ndf16.loc[(ndf16['pri_spec']=='GENERAL PRACTICE'),['NPI']].drop_duplicates().dropna()
c7 = pd.merge(a, b7, on='NPI')

b8 = ndf16.loc[(ndf16['pri_spec']=='PHYSICIAN ASSISTANT'),['NPI']].drop_duplicates().dropna()
c8 = pd.merge(a, b8, on='NPI')


print(scipy.stats.describe(c1.prf_rate))
print(scipy.stats.describe(c2.prf_rate))
print(scipy.stats.describe(c3.prf_rate))
print(scipy.stats.describe(c4.prf_rate))
print(scipy.stats.describe(c5.prf_rate))
print(scipy.stats.describe(c6.prf_rate))
print(scipy.stats.describe(c7.prf_rate))
print(scipy.stats.describe(c8.prf_rate))

print(stats.ttest_ind(c1.prf_rate,c2.prf_rate,equal_var=False))
print(stats.ttest_ind(c1.prf_rate,c3.prf_rate,equal_var=False))
print(stats.ttest_ind(c1.prf_rate,c4.prf_rate,equal_var=False))
print(stats.ttest_ind(c2.prf_rate,c3.prf_rate,equal_var=False))
print(stats.ttest_ind(c2.prf_rate,c4.prf_rate,equal_var=False))
print(stats.ttest_ind(c3.prf_rate,c4.prf_rate,equal_var=False))
print(stats.ttest_ind(c5.prf_rate,c8.prf_rate,equal_var=False))
print(stats.ttest_ind(c2.prf_rate,c7.prf_rate,equal_var=False))
print(stats.ttest_ind(c1.prf_rate,c5.prf_rate,equal_var=False))
print(stats.ttest_ind(c1.prf_rate,c8.prf_rate,equal_var=False))
print(1-58.85706462212487/75.30104623692203)
print(1-59.421996879875195/75.30104623692203)
print(1-62.54601226993865/75.30104623692203)
print(1-63.55072463768116/75.30104623692203)

# Pneumonia Results

# It has been estimated that the total annual excess cost of hospital-treated pneumonia as a primary diagnosis in the elderly fee-for-service Medicare population in 2010 exceeded 7 billion dollars.

# Takeaways from the above cursory comparisons of how clinicians of one primary specialty perform relative to clinicians of another primary specialty regarding how they advise their patients on Pneumonia Vaccination. 

# Physician Assistants (PAs) and Nurse Practitioners (NPs) do similarly well (p = 0.57) but both do significantly poorer than those whose primary specialty was Internal Medicine, with mean score 20% lower

# Surprisingly, those whose primary specialty was Infectious Disease did significantly worse than Internal Medicine (p = 3e-08, Infectious Disease mean score 17% lower) and  Pulmonary Disease (p = 2e-05, Infectious Disease mean score 16% lower), but similarly to General Medicine (p = 0.8).


In [ ]:
female = df1.loc[(df1["Gender"]=="F"), ["Gender"]]
female_accept = df1.loc[(df1["Gender"]=="F") & (df1["Professional accepts Medicare Assignment"]=="Y"), ["NPI","PAC ID","Medical school name","Graduation year"]]
male = df1.loc[(df1["Gender"]=="M"), ["Gender"]]
male_accept = df1.loc[(df1["Gender"]=="M") & (df1["Professional accepts Medicare Assignment"]=="Y"), ["NPI","PAC ID","Medical school name","Graduation year"]]
am = df1.loc[(df1["Gender"]=="M") & (df1["Professional accepts Medicare Assignment"]=="Y"), ["NPI","Medical school name","Graduation year"]]
af = df1.loc[(df1["Gender"]=="F") & (df1["Professional accepts Medicare Assignment"]=="Y"), ["NPI","Medical school name","Graduation year"]]
df2_flu = df2.loc[(df2["Measure Title"]=="Preventive Care and Screening: Influenza Immunization"),["NPI","Measure Performance Rate"]]
df3_pneuvacc = df2.loc[(df2["Measure Title"]=="Pneumonia Vaccination Status for Older Adults"),["NPI","Measure Performance Rate"]]
df4_brcanc = df2.loc[(df2["Measure Title"]=="Breast Cancer Screening"),["NPI","Measure Performance Rate"]]
df5_colocanc = df2.loc[(df2["Measure Title"]=="Colorectal Cancer Screening"),["NPI","Measure Performance Rate"]]
df6_bmi = df2.loc[(df2["Measure Title"]=="Preventive Care and Screening: Body Mass Index (BMI) Screening and Follow-Up Plan"),["NPI","Measure Performance Rate"]]
df7_currmeds = df2.loc[(df2["Measure Title"]=="Documentation of Current Medications in the Medical Record"),["NPI","Measure Performance Rate"]]
df8_eldmal = df2.loc[(df2["Measure Title"]=="Elder Maltreatment Screen and Follow-Up Plan"),["NPI","Measure Performance Rate"]]
df9_tobacco = df2.loc[(df2["Measure Title"]=="Preventive Care and Screening: Tobacco Use: Screening and Cessation Intervention"),["NPI","Measure Performance Rate"]]
df10_osteo = df2.loc[(df2["Measure Title"]=="Screening or Therapy for Osteoporosis for Women Aged 65 Years and Older"),["NPI","Measure Performance Rate"]]
df11_ecg = df2.loc[(df2["Measure Title"]=="Emergency Medicine: 12-Lead Electrocardiogram (ECG) Performed for Non-Traumatic Chest Pain"),["NPI","Measure Performance Rate"]]

brcancm=pd.merge(am, df4_brcanc, on='NPI').drop_duplicates().dropna()
brcancf=pd.merge(af, df4_brcanc, on='NPI').drop_duplicates().dropna()



In [ ]:
colocanc_tobacco=pd.merge(ips15.loc[(ips15['measure_title']=='Colorectal Cancer Screening'),['NPI','prf_rate']],ips15.loc[(ips15['measure_title']=='Preventive Care and Screening: Tobacco Use: Screening and Cessation Intervention'),['NPI','prf_rate']], on='NPI').drop_duplicates().dropna()
flu_pneuvacc=pd.merge(ips15.loc[(ips15['measure_title']=='Preventive Care and Screening: Influenza Immunization'),['NPI','prf_rate']],ips15.loc[(ips15['measure_title']=='Pneumonia Vaccination Status for Older Adults'),['NPI','prf_rate']], on='NPI').drop_duplicates().dropna()
brcancm = pd.merge(ips15.loc[(ips15['measure_title']=='Breast Cancer Screening'),['NPI','prf_rate']],ndf15.loc[(ndf15.gndr=='M'),['NPI','Grd_yr']],on='NPI').drop_duplicates().dropna()
brcancf = pd.merge(ips15.loc[(ips15['measure_title']=='Breast Cancer Screening'),['NPI','prf_rate']],ndf15.loc[(ndf15.gndr=='F'),['NPI','Grd_yr']],on='NPI').drop_duplicates().dropna()
flum = pd.merge(ips15.loc[(ips15['measure_title']=='Preventive Care and Screening: Influenza Immunization'),['NPI','prf_rate']],ndf15.loc[(ndf15.gndr=='M'),['NPI','Grd_yr']],on='NPI').drop_duplicates().dropna()
fluf = pd.merge(ips15.loc[(ips15['measure_title']=='Preventive Care and Screening: Influenza Immunization'),['NPI','prf_rate']],ndf15.loc[(ndf15.gndr=='F'),['NPI','Grd_yr']],on='NPI').drop_duplicates().dropna()

# Figure 4

In [ ]:
ax = colocanc_tobacco.plot.scatter(x='prf_rate_x',y='prf_rate_y',title='Coincidence of performance scores in Colon Cancer Prevention and Smoking Intervention')
ax.set(xlabel='Colon Cancer Screening performance rate', ylabel='Smoking Intervention')
print(scipy.stats.describe(colocanc_tobacco.prf_rate_x))
print(scipy.stats.describe(colocanc_tobacco.prf_rate_y))
print(stats.ttest_ind(colocanc_tobacco.prf_rate_x,colocanc_tobacco.prf_rate_y,equal_var=False))

# Statistical analysis confirms the surprising result seen above. Clinicians who had scores in both categories did an excellent job of counseling their patients with regards to Smoking Cessation (mean value ~95), but those same clinicians did much more poorly when it came to Colorectal Cancer Screening (mean value ~56). Colorectal cancers affect mosly older adults, and the prognosis of those suffering from such cancers drops sharply with disease progression. 

# Figure 5

In [ ]:
fig=plt.figure()
fig.suptitle('Clinician Performance Score for Breast Cancer Screening', fontsize=20)
plt.xlabel('Clinician Graduation Year')
plt.ylabel('Performance Score')
ax1=plt.scatter(brcancm.Grd_yr, brcancm.prf_rate, c='blue', s=0.4, label='Male')
ax2=plt.scatter(brcancf.Grd_yr, brcancf.prf_rate, c='red', s=0.4, label='Female')
plt.gca().legend()
plt.show()

# Figure 6

In [ ]:
fig=plt.figure()
fig.suptitle('Clinician Performance Score for Breast Cancer Screening',fontsize=20)
plt.xlabel('Performance Score for Breast Cancer Screening')
plt.ylabel('Normalized Frequency')
ax1=plt.hist(brcancm.prf_rate, label="Male", bins=dp, color='blue', normed=True)
ax2=plt.hist(brcancf.prf_rate, label="Female", bins=dp, color='red', normed=True, alpha=0.25)
plt.gca().legend()
plt.show()
print(stats.ttest_ind(brcancm.prf_rate,brcancf.prf_rate,equal_var=False))
print(scipy.stats.describe(brcancm.prf_rate))
print(scipy.stats.describe(brcancf.prf_rate))
print(1-57.528294367693945/63.06273836765828)

# Female clinicians perform significantly (p = 1e-31) better at Breast Cancer Screening than do their Male counterparts, with the Male mean score ~9% lower than that of the Female mean score. 

# Figure 7

In [ ]:
fig=plt.figure()
fig.suptitle('Clinician Performance Score for Influenza Administration', fontsize=20)
plt.xlabel('Clinician Graduation Year')
plt.ylabel('Performance Score')
ax1=plt.scatter(flum.Grd_yr, flum.prf_rate, c='blue', s=0.4, label='Male')
ax2=plt.scatter(fluf.Grd_yr, fluf.prf_rate, c='red', s=0.4, label='Female')
plt.gca().legend()
plt.show()

# Figure 8

In [ ]:
fig=plt.figure()
fig.suptitle('Clinician Performance Score for Influenza Immunization',fontsize=20)
plt.xlabel('Score for Influenza Immunization')
plt.ylabel('Normalized Frequency')
ax1=plt.hist(flum.prf_rate, label="Male", bins=dp, color='blue', normed=True)
ax2=plt.hist(fluf.prf_rate, label="Female", bins=dp, color='red', normed=True, alpha=0.25)
plt.gca().legend()
plt.show()
print(stats.ttest_ind(flum.prf_rate,fluf.prf_rate,equal_var=False))
print(scipy.stats.describe(flum.prf_rate))
print(scipy.stats.describe(fluf.prf_rate))
print(1-scipy.stats.describe(flum.prf_rate).mean/scipy.stats.describe(fluf.prf_rate).mean)

# Female clinicians perform significantly (p = 7e-08) better at Influenza Immunization than do their Male counterparts, with the Male mean score ~5% lower than that of the Female mean score.

In [ ]:
dy = np.linspace(1945,2020,16)
a15 = pd.value_counts(ndf15.loc[(ndf15['PQRS']=='Y'),['NPI','Grd_yr']].drop_duplicates().dropna().Grd_yr, bins=dy, sort=False)
b15 = pd.value_counts(ndf15[['NPI','Grd_yr']].drop_duplicates().dropna().Grd_yr, bins=dy, sort=False)
c15 = a15/b15
ax = c15.plot.bar(ylim=(0,0.7), title='Clinician Participation Rate vs Year of Terminal Degree (2015 Data)')
ax.set(xlabel="Year of Clinician's Terminal Degree", ylabel="Clinician participation rate")

# Figure 9

In [ ]:
ax = flu_pneuvacc.plot.scatter(x='prf_rate_x',y='prf_rate_y',title='Coincidence of performance scores in Influenza Immunization and Pneumonia Vaccination')
ax.set(xlabel='Influenza Immunization Score', ylabel='Pneumonia Vaccination Score')
print(scipy.stats.describe(flu_pneuvacc.prf_rate_x))
print(scipy.stats.describe(flu_pneuvacc.prf_rate_y))
print(stats.ttest_ind(flu_pneuvacc.prf_rate_x,flu_pneuvacc.prf_rate_y,equal_var=False))
print(1-scipy.stats.describe(flu_pneuvacc.prf_rate_x).mean/scipy.stats.describe(flu_pneuvacc.prf_rate_y).mean)

# On average clinicians do a significantly worse (p = 4e-237) job counseling their patients regarding Influenza Vaccination (mean score = 50.8, 18% lower than for Pneumonia Vaccination) than they do regarding Pneumonia Vaccination (mean score = 61.8). 

# In Progress, need to vectorize loops(!) and extend into the measure_ID dimension

In [ ]:
## This section collects descriptive statistics on the performance data of clinicians by their primary specialty, generating a
## dataframe in which the columns are specialties and the columns are the various measurement categories covered by the PQRS
## As of now this works on only a single category, and compares clinician primary specialties within that category.
## This needs to be vectorized badly, extremely slow as nested for loops.

spec_list = ndf15.pri_spec.unique()
measure_ID_list = ips15.measure_ID.unique()
spec_mid_descstat = pd.DataFrame(index=np.arange(0, len(spec_list)), columns=measure_ID_list)

for i in range(0, len(measure_ID_list)):
    a = ips15.loc[(ips15['measure_ID']==measure_ID_list[i]),['NPI', 'prf_rate', 'patient_count']].drop_duplicates().dropna()
    for j in range(0, len(spec_list)):
        b = ndf15.loc[(ndf15['pri_spec']==spec_list[j]),['NPI']].drop_duplicates().dropna()
        c = pd.merge(a, b, on='NPI')
        if not c.prf_rate.empty:
            d = scipy.stats.describe(c.prf_rate)
            e = c.patient_count.sum()
            spec_mid_descstat.iat[j,i] = d


In [ ]:
## Ttests between specialties for a given performance measure

spec_mid_ttests = pd.DataFrame(index=spec_list, columns=spec_list)

for i in range(0, len(spec_list)):
    for j in range(0, len(spec_list)):
        if not pd.isnull(spec_mid_descstat.iat[i,0])|pd.isnull(spec_mid_descstat.iat[j,0]) :
            d = scipy.stats.ttest_ind_from_stats(mean1=spec_mid_descstat.iat[i,0].mean, std1=np.sqrt(spec_mid_descstat.iat[i,0].variance), nobs1=spec_mid_descstat.iat[i,0].nobs, mean2=spec_mid_descstat.iat[j,0].mean, std2=np.sqrt(spec_mid_descstat.iat[j,0].variance), nobs2=spec_mid_descstat.iat[j,0].nobs, equal_var=False)
            spec_mid_ttests.iat[j,i] = d
        else:
            spec_mid_ttests.iat[j,i] = d

In [ ]:
#spec_mid_ttests
# Construct heatmap using pvalues?
spec_mid_ttests.iat[3,0].pvalue